In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

import os
from tqdm import tqdm
import itertools

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns

from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import WeightedRandomSampler

torch.backends.cudnn.benchmark = True


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_ds = "../input/tmp-img"
# datatest_ds = "../input/aicup-crops/test"
classes = os.listdir(data_ds)
# classes = classes.sort()
classes.sort(key=lambda x:int(x[6:-4]))
classes.sort(key=lambda x:int(x[0:5]))
print(classes)

['00351_52.jpg', '00351_75.jpg', '00351_95.jpg', '00351_114.jpg', '00351_129.jpg', '00351_167.jpg', '00352_34.jpg', '00352_99.jpg', '00352_122.jpg', '00352_145.jpg', '00352_182.jpg', '00352_225.jpg', '00352_253.jpg', '00352_303.jpg', '00352_324.jpg', '00352_356.jpg', '00352_402.jpg', '00352_447.jpg', '00352_467.jpg', '00352_496.jpg', '00352_539.jpg', '00352_562.jpg', '00352_593.jpg', '00352_628.jpg', '00352_668.jpg', '00352_702.jpg', '00352_719.jpg', '00352_738.jpg', '00352_753.jpg', '00352_771.jpg', '00352_782.jpg', '00352_803.jpg', '00352_817.jpg', '00352_853.jpg', '00352_880.jpg', '00352_911.jpg', '00352_951.jpg', '00352_993.jpg', '00352_1015.jpg', '00352_1043.jpg', '00352_1083.jpg', '00352_1096.jpg', '00352_1132.jpg', '00352_1163.jpg', '00352_1177.jpg', '00352_1205.jpg', '00352_1245.jpg', '00352_1269.jpg', '00353_17.jpg', '00353_39.jpg', '00353_75.jpg', '00353_101.jpg', '00353_114.jpg', '00353_144.jpg', '00353_169.jpg', '00353_193.jpg', '00353_232.jpg', '00353_250.jpg', '00353_282.

In [3]:
df = pd.read_csv("/kaggle/input/badmin/all_hit_labels.csv", encoding = 'utf-8')
df.iloc[3250:3700]
# df

,VideoName,ShotSeq,HitFrame,Hitter,RoundHead,Backhand,BallHeight,LandingX,LandingY,HitterLocationX,HitterLocationY,DefenderLocationX,DefenderLocationY,BallType,Winner
3250,350,2,35,A,2,2,2,723,457,595,355,642,498,6,X
3251,350,3,51,B,2,2,2,643,349,693,509,600,344,6,X
3252,350,4,70,A,2,1,2,560,488,589,351,691,504,9,X
3253,350,5,82,B,2,1,1,689,327,642,498,584,368,4,X
3254,350,6,90,A,2,1,2,731,477,571,343,631,527,4,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,393,4,101,B,2,2,2,760,276,751,425,687,346,8,X
3696,393,5,158,A,2,2,1,861,471,732,271,723,486,7,X
3697,393,6,176,B,2,2,2,527,367,795,486,730,285,6,X
3698,393,7,207,A,2,2,2,392,555,557,351,639,464,8,X


In [4]:
import shutil

shutil.rmtree('/kaggle/temp/')

valid_num = 25

y_t = os.path.join('/kaggle/temp/train', 'y')
y_v = os.path.join('/kaggle/temp/valid', 'y')
n_t = os.path.join('/kaggle/temp/train', 'n')
n_v = os.path.join('/kaggle/temp/valid', 'n')
os.makedirs(y_t)
os.makedirs(y_v)
os.makedirs(n_t)
os.makedirs(n_v)


y = []
n = []
for aclass in classes:
    mask1 = df['VideoName'] == int(aclass[0:5])
    mask2 = df['HitFrame'] == int(aclass[6:-4])
    tmp = int(df[(mask1 & mask2)].RoundHead)
    if(tmp==1):
        y.append(aclass)
    else:
        n.append(aclass)

for f in y[valid_num:]:
    src = os.path.join(data_ds, f)
    dst = os.path.join(y_t, f)
    shutil.copy(src, dst)
for f in y[:valid_num]:
    src = os.path.join(data_ds, f)
    dst = os.path.join(y_v, f)
    shutil.copy(src, dst)
for f in n[valid_num:]:
    src = os.path.join(data_ds, f)
    dst = os.path.join(n_t, f)
    shutil.copy(src, dst)
for f in n[:valid_num]:
    src = os.path.join(data_ds, f)
    dst = os.path.join(n_v, f)
    shutil.copy(src, dst)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/temp/'

In [ ]:
transforms = T.Compose([
#     T.Resize((360, 360)),
#     T.RandomHorizontalFlip(p=0.5),
#     T.RandomEqualize(p=1),
    T.ToTensor(),
#     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
train_ds = ImageFolder('/kaggle/temp/train', transform = transforms)
val_ds = ImageFolder('/kaggle/temp/valid', transform = transforms)

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

In [ ]:
len(train_ds), len(val_ds)

In [ ]:
batch_size = 16

In [ ]:
train_y = [label for img, label in train_ds.samples ]

class_sample_count = np.array([len(np.where(train_y == t)[0]) for t in np.unique(train_y)])
class_sample_count


In [ ]:
# weight = 1. / class_sample_count
# samples_weight = torch.DoubleTensor([weight[t] for t in train_y])
# sampler = WeightedRandomSampler(samples_weight, 20000, replacement=False)

In [ ]:
train_dl = DataLoader(train_ds, batch_size, num_workers = 2,  pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers = 2, pin_memory=True)

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)

跑GPU

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
#改batch

In [ ]:
device = get_default_device()
device

Train model的函式

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def predict_step(self, batch, y_pred, y_true):
        images, labels = batch 
        out = self(images)  # Generate predictions
        _, preds = torch.max(out, 1)
        loss = F.cross_entropy(out, labels)   # Calculate loss
        y_pred.extend(preds.view(-1).detach().cpu().numpy())       # 將preds預測結果detach出來，並轉成numpy格式       
        y_true.extend(labels.view(-1).detach().cpu().numpy())      # target是ground-truth的label
        #acc = accuracy(out, labels)           # Calculate accuracy
        return y_pred, y_true
        #return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        torch.save(self.network.state_dict(), '/kaggle/working/model-' + str(epoch) + '.pth')
        print("Epoch {}: train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch+1, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
@torch.no_grad()
def predict(model, val_loader, y_pred, y_true):
    model.eval()
    for batch in val_loader:
        y_pred, y_true=model.predict_step(batch, y_pred, y_true)
    #outputs = [model.validation_step(batch) for batch in tqdm(val_loader)]
    return y_pred, y_true

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, y_pred, y_true, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        predict(model, val_dl, y_pred, y_true )
        target_names = ['y', 'n']
        cnf_matrix = confusion_matrix(y_true, y_pred)
        per_cls_acc = cnf_matrix.diagonal()/cnf_matrix.sum(axis=0)                    # https://stackoverflow.com/a/53824126/13369757
        print(target_names)
        print(per_cls_acc)                                                          #顯示每個class的Accuracy
        print("Plot confusion matrix")

        # 開始繪製混淆矩陣
        df_cm = pd.DataFrame(cnf_matrix, target_names, target_names)     # https://sofiadutta.github.io/datascience-ipynbs/pytorch/Image-Classification-using-PyTorch.html
        plt.figure(figsize = (2,2))
        sns.heatmap(df_cm, annot=True, fmt="d", cmap='Blues')
        plt.xlabel("prediction")
        plt.ylabel("label (ground truth)")
        y_pred=[]
        y_true=[]
    return history

In [ ]:
from torch.cuda.amp import autocast
class EfficientNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = timm.create_model('efficientnet_b0', pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier.in_features
        self.network.classifier = nn.Linear(num_ftrs, 2)
    
    @autocast()
    def forward(self, xb):
        return self.network(xb)

model = EfficientNet()


In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

In [ ]:
model = to_device(EfficientNet(), device)
#evaluate(model, val_dl)

In [ ]:
y_pred=[]
y_true=[]

In [ ]:
num_epochs = 1
opt_func = torch.optim.AdamW
lr = 3e-4
history = fit(num_epochs, lr, model, train_dl, val_dl, y_pred, y_true, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

In [ ]:
# def predict_image(img, model):
#     # Convert to a batch of 1
#     xb = to_device(img.unsqueeze(0), device)
#     # Get predictions from model
#     yb = model(xb)
#     # Pick index with highest probability
#     prob, preds  = torch.max(yb, dim=1)
#     # Retrieve the class label
#     return dataset.classes[preds[0].item()]

測試test

In [ ]:
def test(model, dataloader):
    model.eval()
    pred_list =[]
    pred_id_list=[]
    with torch.no_grad():
        with tqdm(dataloader,unit='batch',desc='Test') as tqdm_loader:
            for idx, (imgid, img) in enumerate(tqdm_loader):

                img = img.to(device=cfg['device'])
                pred = model(img).detach().cpu().argmax(dim=1)

                pred_list.append(pred)
                pred_id_list.append(imgid)
    pred_list = np.concatenate(pred_list,axis=0)
    pred_id_list =  np.concatenate(pred_id_list,axis=0)
    return pred_list,pred_id_list
pred_list,pred_id_list = test(model,test_dl)

In [ ]:
pred_id_list = [imageid+'.jpg' for imageid in pred_id_list]

In [ ]:
submit_df  = pd.DataFrame(columns=upload_sample_df.columns)
submit_df['image_filename'] = pred_id_list
submit_df['label'] = pred_list
submit_df.to_csv('submit.csv',index=False)
print(submit_df.head(10))